In [1]:
import os
import concurrent.futures
import json

import httpx
import pandas as pd

In [2]:
sample_file = "../extras/annotated_data.tsv"
predictive_test_name = "SimpleMatchApproach"

In [3]:
df = pd.read_csv(sample_file, sep="\t")
df.head()

,text,diagnostic
0,acho que tenho alguma coisa na gargant que ta ...,negativo
1,Ando com muitas dores nas costas após o trabalho.,negativo
2,Ando preocupada com a quantidade de cabelo que...,negativo
3,Ando sentindo um desconforto abdominal após as...,negativo
4,As pernas estão pesadas e sinto formigamento n...,negativo


In [4]:
with open('../setup.py') as f:
    contents = f.read()

start = contents.find('__version__') + len('__version__')
end = contents.find('\n', start)
current_version = contents[start:end].replace('=', '').strip()
current_version

'"1.1.0"'

In [5]:
def make_request(name, text, extra_args):
    url = 'http://localhost:8000/pre-diagnostic/process-diagnostic'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    data = {
        'name': name,
        'text': text,
        'extra_args': extra_args
    }
    httpx.post(url, headers=headers, data=json.dumps(data))
    return True

make_request("joao", "um teste",  extra_args = {'version': current_version, 'expected': 'negativo'})

True

In [6]:
def make_request_for_row(row, version=current_version):
    extra_args = {
        'version': version,
        'expected': row['diagnostic']
    }
    return make_request(predictive_test_name, row['text'], extra_args)

with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
    executor.map(executor.map(make_request_for_row, df.to_dict('records')))
